# %% [markdown]

 Task 6D: pandas vs SQL
 Name: Ocean
 Student Number: s223503101
 Email: s223503101@deakin.edu.au
 Undergraduate (SIT220)

# %% [markdown]

 Introduction
 This report aims to replicate SQL queries using pandas, working with the nycflights13 dataset.
 It involves creating an SQLite database, importing CSVs, and writing equivalent pandas code
 for various SQL queries while ensuring output parity using pd.testing.assert_frame_equal.

In [ ]:
# %%

import pandas as pd
import sqlite3

# Load datasets
planes = pd.read_csv(r"C:\Users\sumit\Downloads\New folder (2)\nycflights13_planes.csv.gz", comment="#")
flights = pd.read_csv(r"C:\Users\sumit\Downloads\New folder (2)\nycflights13_airports.csv.gz", comment="#")
airports = pd.read_csv(r"C:\Users\sumit\Downloads\New folder (2)\nycflights13_flights.csv.gz", comment="#")
airlines = pd.read_csv(r"C:\Users\sumit\Downloads\New folder (2)\nycflights13_airlines.csv.gz", comment="#")
weather = pd.read_csv(r"C:\Users\sumit\Downloads\New folder (2)\nycflights13_weather.csv.gz", comment="#")

# Create SQLite connection
conn = sqlite3.connect("nycflights.db")

# Export to SQLite
planes.to_sql("planes", conn, if_exists="replace", index=False)
flights.to_sql("flights", conn, if_exists="replace", index=False)
airports.to_sql("airports", conn, if_exists="replace", index=False)
airlines.to_sql("airlines", conn, if_exists="replace", index=False)
weather.to_sql("weather", conn, if_exists="replace", index=False)

26130

# %% [markdown]

 ## Query 1: Unique Engine Types

 This query retrieves all the unique types of aircraft engines in the dataset.
 It's like asking, “What different engine types are present in all the planes?”

 We use SQL's `SELECT DISTINCT` and pandas' `drop_duplicates()` to get this result.
 The output from both methods is compared to confirm correctness.

In [ ]:
# %%

# SQL query 
query1_sql = pd.read_sql_query("SELECT DISTINCT engine FROM planes", conn)

# Equivalent pandas query
query1_pd = planes[["engine"]].drop_duplicates()

# Print the pandas output
print("Pandas Query 1 Result:\n", query1_pd.head())

# Check if both outputs match
pd.testing.assert_frame_equal(
    query1_sql.sort_values("engine").reset_index(drop=True),
    query1_pd.sort_values("engine").reset_index(drop=True)
)

# Final confirmation
print(" Query 1: SQL and pandas results match.")

Pandas Query 1 Result:
             engine
0        Turbo-fan
51       Turbo-jet
424  Reciprocating
686        4 Cycle
811    Turbo-shaft
 Query 1: SQL and pandas results match.


# %% [markdown]

 ## Query 2: Unique Type-Engine Combinations

 This query retrieves all unique combinations of aircraft `type` and `engine`.
 It's like asking, “Which types of planes are paired with which engine types?”

 We use SQL's `SELECT DISTINCT type, engine` and pandas' `drop_duplicates()` on both columns.
 The outputs are compared to ensure both methods yield the same result.

In [ ]:
# %%

# SQL query 
query2_sql = pd.read_sql_query("SELECT DISTINCT type, engine FROM planes", conn)

# Equivalent pandas query
query2_pd = planes[["type", "engine"]].drop_duplicates()

# Print the pandas output
print("Pandas Query 2 Result:\n", query2_pd.head())

# Validate equivalence
pd.testing.assert_frame_equal(
    query2_sql.sort_values(by=["type", "engine"]).reset_index(drop=True),
    query2_pd.sort_values(by=["type", "engine"]).reset_index(drop=True)
)

# Final confirmation
print(" Query 2: SQL and pandas results match.")

Pandas Query 2 Result:
                          type         engine
0     Fixed wing multi engine      Turbo-fan
51    Fixed wing multi engine      Turbo-jet
424  Fixed wing single engine  Reciprocating
427   Fixed wing multi engine  Reciprocating
686  Fixed wing single engine        4 Cycle
 Query 2: SQL and pandas results match.


# %% [markdown]

 ## Query 3: Count of Planes by Engine Type

 This query counts how many planes use each type of engine.
 Imagine you’re saying, “Tell me how many planes use turbofan, turbojet, etc.”

 SQL uses `GROUP BY engine` and `COUNT(*)`. In pandas, we use `groupby()` and `size()`.
 We reorder columns and validate the result.

In [ ]:
# %%

# SQL query
query3_sql = pd.read_sql_query("SELECT COUNT(*) as count, engine FROM planes GROUP BY engine", conn)
query3_sql = query3_sql[["engine", "count"]]  # ensure column order matches pandas

# pandas equivalent
query3_pd = planes.groupby("engine").size().reset_index(name="count")

# Print pandas output
print("Pandas Query 3 Result:\n", query3_pd.head())

# Validation
pd.testing.assert_frame_equal(
    query3_sql.sort_values("engine").reset_index(drop=True),
    query3_pd.sort_values("engine").reset_index(drop=True)
)

# Final confirmation
print(" Query 3: SQL and pandas results match.")

Pandas Query 3 Result:
           engine  count
0        4 Cycle      2
1  Reciprocating     28
2      Turbo-fan   2750
3      Turbo-jet    535
4     Turbo-prop      2
 Query 3: SQL and pandas results match.


# %% [markdown]

 ## Query 4: Count of Planes by Engine and Type

 This query counts how many planes exist for each unique combination of engine type and aircraft type.
 It answers: “How many planes are there for each engine–type pairing?”

 SQL uses `GROUP BY engine, type`, while pandas uses `groupby(["engine", "type"])`.
 The output is then compared after sorting and resetting the index.

In [ ]:
# %%

# SQL query
query4_sql = pd.read_sql_query(
    "SELECT COUNT(*) as count, engine, type FROM planes GROUP BY engine, type",
    conn
)
query4_sql = query4_sql[["engine", "type", "count"]]  # reorder to match pandas

# pandas equivalent
query4_pd = planes.groupby(["engine", "type"]).size().reset_index(name="count")

# Print pandas output
print("Pandas Query 4 Result:\n", query4_pd.head())

# Validation
pd.testing.assert_frame_equal(
    query4_sql.sort_values(by=["engine", "type"]).reset_index(drop=True),
    query4_pd.sort_values(by=["engine", "type"]).reset_index(drop=True)
)

# Final confirmation
print(" Query 4: SQL and pandas results match.")

Pandas Query 4 Result:
           engine                      type  count
0        4 Cycle  Fixed wing single engine      2
1  Reciprocating   Fixed wing multi engine      5
2  Reciprocating  Fixed wing single engine     23
3      Turbo-fan   Fixed wing multi engine   2750
4      Turbo-jet   Fixed wing multi engine    535
 Query 4: SQL and pandas results match.


# %% [markdown]

 ## Query 5: Plane Age Statistics by Engine and Manufacturer

 This query shows the **oldest**, **average**, and **newest** manufacturing year of planes,
 grouped by their engine type and manufacturer.

 It answers the question: “For each engine-manufacturer combo, what's the range of manufacturing years?”

 SQL uses `MIN`, `AVG`, `MAX` with `GROUP BY`, while pandas uses `.agg()` with grouped data.

In [ ]:
# %%

# SQL query
query5_sql = pd.read_sql_query("""
    SELECT MIN(year) AS min_year, AVG(year) AS avg_year, MAX(year) AS max_year, engine, manufacturer
    FROM planes
    GROUP BY engine, manufacturer
""", conn)

# Reorder columns to match pandas result
query5_sql = query5_sql[["engine", "manufacturer", "min_year", "avg_year", "max_year"]]

# pandas equivalent
query5_pd = planes.groupby(["engine", "manufacturer"]).agg({
    "year": ["min", "mean", "max"]
}).reset_index()

# Flatten multi-level column names
query5_pd.columns = ["engine", "manufacturer", "min_year", "avg_year", "max_year"]

# Print pandas output
print("Pandas Query 5 Result:\n", query5_pd.head())

# Validation
pd.testing.assert_frame_equal(
    query5_sql.sort_values(by=["engine", "manufacturer"]).reset_index(drop=True),
    query5_pd.sort_values(by=["engine", "manufacturer"]).reset_index(drop=True)
)

# Final confirmation
print(" Query 5: SQL and pandas results match.")

Pandas Query 5 Result:
           engine           manufacturer  min_year  avg_year  max_year
0        4 Cycle                 CESSNA    1975.0    1975.0    1975.0
1        4 Cycle            JOHN G HESS       NaN       NaN       NaN
2  Reciprocating  AMERICAN AIRCRAFT INC       NaN       NaN       NaN
3  Reciprocating     AVIAT AIRCRAFT INC    2007.0    2007.0    2007.0
4  Reciprocating          BARKER JACK L       NaN       NaN       NaN
 Query 5: SQL and pandas results match.


# %% [markdown]

 ## Query 6: Planes with Non-Null Speed

 This query filters the dataset to return only those planes that have a recorded (non-null) `speed` value.
 In simple terms, it's like asking: “Which planes have a known speed?”

 SQL uses `WHERE speed IS NOT NULL`, while pandas uses `.notna()` to achieve the same effect.

In [ ]:
# %%

# SQL query
query6_sql = pd.read_sql_query("SELECT * FROM planes WHERE speed IS NOT NULL", conn)

# pandas equivalent
query6_pd = planes[planes["speed"].notna()].reset_index(drop=True)

# Print pandas output
print("Pandas Query 6 Result:\n", query6_pd.head())

# Validation
pd.testing.assert_frame_equal(
    query6_sql.sort_values("tailnum").reset_index(drop=True),
    query6_pd.sort_values("tailnum").reset_index(drop=True)
)

# Final confirmation
print(" Query 6: SQL and pandas results match.")

Pandas Query 6 Result:
   tailnum    year                      type manufacturer      model  engines  \
0  N201AA  1959.0  Fixed wing single engine       CESSNA        150        1   
1  N202AA  1980.0   Fixed wing multi engine       CESSNA       421C        2   
2  N350AA  1980.0   Fixed wing multi engine        PIPER  PA-31-350        2   
3  N364AA  1973.0   Fixed wing multi engine       CESSNA       310Q        2   
4  N378AA  1963.0  Fixed wing single engine       CESSNA       172E        1   

   seats  speed         engine  
0      2   90.0  Reciprocating  
1      8   90.0  Reciprocating  
2      8  162.0  Reciprocating  
3      6  167.0  Reciprocating  
4      4  105.0  Reciprocating  
 Query 6: SQL and pandas results match.


# %% [markdown]

 ## Query 7: Planes with 150–210 Seats and Manufactured After 2010

 This query filters the dataset to include only planes that:
 - have a seating capacity between 150 and 210 (inclusive), **and**
 - were manufactured in 2011 or later.

 It's like saying: “Give me all fairly recent planes that are mid-sized.”

 SQL uses `BETWEEN` and `>=`. In pandas, we use `.between()` and logical conditions.

In [ ]:
# %%

# SQL query
query7_sql = pd.read_sql_query("""
    SELECT tailnum FROM planes
    WHERE seats BETWEEN 150 AND 210 AND year >= 2011
""", conn)

# pandas equivalent
query7_pd = planes[
    (planes["seats"].between(150, 210)) & (planes["year"] >= 2011)
][["tailnum"]]

# Print pandas output
print("Pandas Query 7 Result:\n", query7_pd.head())

# Validation
pd.testing.assert_frame_equal(
    query7_sql.sort_values("tailnum").reset_index(drop=True),
    query7_pd.sort_values("tailnum").reset_index(drop=True)
)

# Final confirmation
print(" Query 7: SQL and pandas results match.")

Pandas Query 7 Result:
     tailnum
215  N150UW
216  N151UW
218  N152UW
221  N153UW
223  N154UW
 Query 7: SQL and pandas results match.


# %% [markdown]

 ## Query 8: Large Planes from Specific Manufacturers

 This query selects planes made by **BOEING**, **AIRBUS**, or **EMBRAER**, and with more than 390 seats.

 It's like asking: “Show me the very large planes from these three major manufacturers.”

 SQL uses `IN (...)` and a condition on `seats > 390`. In pandas, we use `.isin()` and a logical `&`.

In [ ]:
# %%

# SQL query
query8_sql = pd.read_sql_query("""
    SELECT tailnum, manufacturer, seats FROM planes
    WHERE manufacturer IN ('BOEING', 'AIRBUS', 'EMBRAER') AND seats > 390
""", conn)

# pandas equivalent
query8_pd = planes[
    planes["manufacturer"].isin(["BOEING", "AIRBUS", "EMBRAER"]) & (planes["seats"] > 390)
][["tailnum", "manufacturer", "seats"]]

# Print pandas output
print("Pandas Query 8 Result:\n", query8_pd.head())

# Validation
pd.testing.assert_frame_equal(
    query8_sql.sort_values("tailnum").reset_index(drop=True),
    query8_pd.sort_values("tailnum").reset_index(drop=True)
)

# Final confirmation
print(" Query 8: SQL and pandas results match.")

Pandas Query 8 Result:
      tailnum manufacturer  seats
439   N206UA       BOEING    400
484   N228UA       BOEING    400
577   N272AT       BOEING    400
1708  N57016       BOEING    400
2109  N670US       BOEING    450
 Query 8: SQL and pandas results match.


# %% [markdown]

 ## Query 9: Distinct Year–Seat Combinations After 2011

 This query retrieves all **unique combinations** of `year` and `seats` for planes manufactured in **2012 or later**,
 sorted first by **year in ascending order** and then by **seats in descending order**.

 It's like saying: “Among newer planes, what unique combinations of year and seat count exist?”

 SQL uses `SELECT DISTINCT` and `ORDER BY year ASC, seats DESC`.
 pandas uses `drop_duplicates()` and `sort_values()`.

In [ ]:
# %%

# SQL query
query9_sql = pd.read_sql_query("""
    SELECT DISTINCT year, seats FROM planes
    WHERE year >= 2012
    ORDER BY year ASC, seats DESC
""", conn)

# pandas equivalent
query9_pd = planes[planes["year"] >= 2012][["year", "seats"]].drop_duplicates()
query9_pd = query9_pd.sort_values(by=["year", "seats"], ascending=[True, False]).reset_index(drop=True)

# Print pandas output
print("Pandas Query 9 Result:\n", query9_pd.head())

# Validation
pd.testing.assert_frame_equal(query9_sql, query9_pd)

# Final confirmation
print(" Query 9: SQL and pandas results match.")

Pandas Query 9 Result:
      year  seats
0  2012.0    379
1  2012.0    377
2  2012.0    260
3  2012.0    222
4  2012.0    200
 Query 9: SQL and pandas results match.


# %% [markdown]

 ## Query 10: Distinct Year–Seat Combinations (Seats DESC, Year ASC)

 This query is similar to Query 9 but with a different **sort order**.
 It still finds unique combinations of `year` and `seats` for planes made from **2012 onwards**,
 but now sorts results by **seats in descending order**, then **year in ascending order**.

 It's like asking: “Among newer planes, list all distinct year-seat combos, starting with the biggest planes.”

In [ ]:
# %%

# SQL query
query10_sql = pd.read_sql_query("""
    SELECT DISTINCT year, seats FROM planes
    WHERE year >= 2012
    ORDER BY seats DESC, year ASC
""", conn)

# pandas equivalent
query10_pd = planes[planes["year"] >= 2012][["year", "seats"]].drop_duplicates()
query10_pd = query10_pd.sort_values(by=["seats", "year"], ascending=[False, True]).reset_index(drop=True)

# Print pandas output
print("Pandas Query 10 Result:\n", query10_pd.head())

# Validation
pd.testing.assert_frame_equal(query10_sql, query10_pd)

# Final confirmation
print(" Query 10: SQL and pandas results match.")

Pandas Query 10 Result:
      year  seats
0  2012.0    379
1  2013.0    379
2  2012.0    377
3  2013.0    377
4  2012.0    260
 Query 10: SQL and pandas results match.


# %% [markdown]

 ## Query 11: Count of Large Planes by Manufacturer

 This query counts how many planes with **more than 200 seats** each manufacturer has.

 It answers the question: “Which manufacturers have large aircraft in the dataset, and how many?”

 SQL uses a `WHERE` clause followed by `GROUP BY manufacturer`.
 pandas applies a filter first, then `groupby()` and `size()`.

In [ ]:
# %%

# SQL query
query11_sql = pd.read_sql_query("""
    SELECT manufacturer, COUNT(*) FROM planes
    WHERE seats > 200
    GROUP BY manufacturer
""", conn)

# pandas equivalent
query11_pd = planes[planes["seats"] > 200].groupby("manufacturer").size().reset_index(name="COUNT(*)")

# Print pandas output
print("Pandas Query 11 Result:\n", query11_pd.head())

# Validation
pd.testing.assert_frame_equal(
    query11_sql.sort_values("manufacturer").reset_index(drop=True),
    query11_pd.sort_values("manufacturer").reset_index(drop=True)
)

# Final confirmation
print(" Query 11: SQL and pandas results match.")

Pandas Query 11 Result:
        manufacturer  COUNT(*)
0            AIRBUS        66
1  AIRBUS INDUSTRIE         4
2            BOEING       225
 Query 11: SQL and pandas results match.


# %% [markdown]

 ## Query 12: Manufacturers with More Than 10 Planes

 This query returns only those manufacturers that appear more than 10 times in the dataset.
 In SQL, this is done using a `GROUP BY` with a `HAVING COUNT(*) > 10` clause.

 In pandas, we first `groupby()` and filter groups where the count exceeds 10,
 then count again by manufacturer.

In [ ]:
# %%

# SQL query
query12_sql = pd.read_sql_query("""
    SELECT manufacturer, COUNT(*) FROM planes
    GROUP BY manufacturer
    HAVING COUNT(*) > 10
""", conn)

# pandas equivalent
query12_pd = planes.groupby("manufacturer").filter(lambda x: len(x) > 10)
query12_pd = query12_pd.groupby("manufacturer").size().reset_index(name="COUNT(*)")

# Print pandas output
print("Pandas Query 12 Result:\n", query12_pd.head())

# Validation
pd.testing.assert_frame_equal(
    query12_sql.sort_values("manufacturer").reset_index(drop=True),
    query12_pd.sort_values("manufacturer").reset_index(drop=True)
)

# Final confirmation
print(" Query 12: SQL and pandas results match.")

Pandas Query 12 Result:
        manufacturer  COUNT(*)
0            AIRBUS       336
1  AIRBUS INDUSTRIE       400
2            BOEING      1630
3    BOMBARDIER INC       368
4           EMBRAER       299
 Query 12: SQL and pandas results match.


# %% [markdown]

 ## Query 13: Manufacturers with >10 Large Planes (Seats > 200)

 This query filters for manufacturers who have **more than 10 planes** that each have **more than 200 seats**.

 It combines two conditions:
 1. Filter planes with `seats > 200`
 2. Group by manufacturer and only keep those with a **count greater than 10**

 This is a combined `WHERE` and `HAVING` clause in SQL. pandas handles it using a `filter()` on groups.

In [ ]:
# %%

# SQL query
query13_sql = pd.read_sql_query("""
    SELECT manufacturer, COUNT(*) FROM planes
    WHERE seats > 200
    GROUP BY manufacturer
    HAVING COUNT(*) > 10
""", conn)

# pandas equivalent
query13_pd = planes[planes["seats"] > 200]
query13_pd = query13_pd.groupby("manufacturer").filter(lambda x: len(x) > 10)
query13_pd = query13_pd.groupby("manufacturer").size().reset_index(name="COUNT(*)")

# Print pandas output
print("Pandas Query 13 Result:\n", query13_pd.head())

# Validation
pd.testing.assert_frame_equal(
    query13_sql.sort_values("manufacturer").reset_index(drop=True),
    query13_pd.sort_values("manufacturer").reset_index(drop=True)
)

# Final confirmation
print(" Query 13: SQL and pandas results match.")

Pandas Query 13 Result:
   manufacturer  COUNT(*)
0       AIRBUS        66
1       BOEING       225
 Query 13: SQL and pandas results match.


# %% [markdown]

 ## Query 14: Top 10 Manufacturers by Number of Planes

 This query finds the top 10 aircraft manufacturers based on the number of planes they have in the dataset.

 It answers: “Which manufacturers appear most frequently, and how many planes do they have?”

 SQL uses `GROUP BY`, `COUNT(*)`, `ORDER BY DESC`, and `LIMIT 10`.
 In pandas, we group by manufacturer, count, sort in descending order, and take the top 10 rows.

In [ ]:
# %%

# SQL query
query14_sql = pd.read_sql_query("""
    SELECT manufacturer, COUNT(*) AS howmany FROM planes
    GROUP BY manufacturer
    ORDER BY howmany DESC
    LIMIT 10
""", conn)

# pandas equivalent
query14_pd = planes.groupby("manufacturer").size().reset_index(name="howmany")
query14_pd = query14_pd.sort_values("howmany", ascending=False).head(10).reset_index(drop=True)

# Print pandas output
print("Pandas Query 14 Result:\n", query14_pd)

# Validation
pd.testing.assert_frame_equal(query14_sql, query14_pd)

# Final confirmation
print(" Query 14: SQL and pandas results match.")

Pandas Query 14 Result:
                     manufacturer  howmany
0                         BOEING     1630
1               AIRBUS INDUSTRIE      400
2                 BOMBARDIER INC      368
3                         AIRBUS      336
4                        EMBRAER      299
5              MCDONNELL DOUGLAS      120
6  MCDONNELL DOUGLAS AIRCRAFT CO      103
7  MCDONNELL DOUGLAS CORPORATION       14
8                         CESSNA        9
9                       CANADAIR        9
 Query 14: SQL and pandas results match.


# %% [markdown]

 ## Query 15: Merge Flight Data with Plane Details

 This query performs a **left join** between the `flights` and `planes` tables on the `tailnum` field.

 The goal is to enrich each flight record with additional aircraft details: `year`, `speed`, and `seats`.

 This is like saying: “Add plane information (year, speed, seats) to each flight, matched by tail number.”

 SQL uses `LEFT JOIN`. In pandas, we use `pd.merge(..., how="left")`.

In [ ]:
# %%

# Reload just in case to ensure fresh column names
flights = pd.read_csv(r"C:\Users\sumit\Downloads\New folder (2)\nycflights13_flights.csv.gz", comment="#")
planes = pd.read_csv(r"C:\Users\sumit\Downloads\New folder (2)\nycflights13_planes.csv.gz", comment="#")

# Clean column names
flights.columns = flights.columns.str.strip()
planes.columns = planes.columns.str.strip()

# Perform the left join
merged_15 = pd.merge(
    flights,
    planes[["tailnum", "year", "speed", "seats"]],
    on="tailnum",
    how="left"
)

# Rename columns to avoid conflicts
merged_15.rename(columns={
    "year_x": "year",          # flights year (if needed)
    "year_y": "plane_year",
    "speed": "plane_speed",
    "seats": "plane_seats"
}, inplace=True)

# Ensure proper column order: all original flight columns + new ones
columns_to_keep_15 = list(flights.columns) + ["plane_year", "plane_speed", "plane_seats"]
merged_15 = merged_15[columns_to_keep_15]

# Preview the result
print("Pandas Query 15 Result (first 5 rows):\n", merged_15.head())

# Final confirmation
print(" Query 15: pandas-only JOIN completed successfully.")

Pandas Query 15 Result (first 5 rows):
    year  month  day  dep_time  sched_dep_time  dep_delay  arr_time  \
0  2013      1    1     517.0             515        2.0     830.0   
1  2013      1    1     533.0             529        4.0     850.0   
2  2013      1    1     542.0             540        2.0     923.0   
3  2013      1    1     544.0             545       -1.0    1004.0   
4  2013      1    1     554.0             600       -6.0     812.0   

   sched_arr_time  arr_delay carrier  ...  origin dest air_time distance  \
0             819       11.0      UA  ...     EWR  IAH    227.0     1400   
1             830       20.0      UA  ...     LGA  IAH    227.0     1416   
2             850       33.0      AA  ...     JFK  MIA    160.0     1089   
3            1022      -18.0      B6  ...     JFK  BQN    183.0     1576   
4             837      -25.0      DL  ...     LGA  ATL    116.0      762   

   hour  minute            time_hour  plane_year plane_speed  plane_seats  
0     

# %% [markdown]

 ## Query 16: Join Flights' Carrier-Tailnum with Planes and Airlines

 This query performs a **two-step join**:
 1. It starts by getting all **distinct combinations** of `carrier` and `tailnum` from the `flights` table.
 2. Then it joins this result with `planes` on `tailnum` and with `airlines` on `carrier`.

 It's like asking: “Give me full details about each unique carrier-aircraft combination.”

 In pandas, we:
 - use `drop_duplicates()` to get unique pairs,
 - then merge with `planes` and `airlines` using `pd.merge`.

In [ ]:
# %%

# Clean column names
airlines.columns = airlines.columns.str.strip()
planes.columns = planes.columns.str.strip()
flights.columns = flights.columns.str.strip()

# Step 1: Get distinct (carrier, tailnum) pairs
cartail = flights[["carrier", "tailnum"]].drop_duplicates()

# Step 2: Join with planes on tailnum
merged_16 = pd.merge(cartail, planes, on="tailnum", how="inner")

# Step 3: Join with airlines on carrier
merged_16 = pd.merge(merged_16, airlines, on="carrier", how="inner")

# Print result preview
print("Pandas Query 16 Result (first 5 rows):\n", merged_16.head())

# Final confirmation
print(" Query 16: pandas-only JOIN with planes and airlines completed successfully.")

Pandas Query 16 Result (first 5 rows):
   carrier tailnum    year                     type manufacturer     model  \
0      UA  N14228  1999.0  Fixed wing multi engine       BOEING   737-824   
1      UA  N24211  1998.0  Fixed wing multi engine       BOEING   737-824   
2      AA  N619AA  1990.0  Fixed wing multi engine       BOEING   757-223   
3      B6  N804JB  2012.0  Fixed wing multi engine       AIRBUS  A320-232   
4      DL  N668DN  1991.0  Fixed wing multi engine       BOEING   757-232   

   engines  seats  speed     engine                    name  
0        2    149    NaN  Turbo-fan   United Air Lines Inc.  
1        2    149    NaN  Turbo-fan   United Air Lines Inc.  
2        2    178    NaN  Turbo-fan  American Airlines Inc.  
3        2    200    NaN  Turbo-fan         JetBlue Airways  
4        2    178    NaN  Turbo-fan    Delta Air Lines Inc.  
 Query 16: pandas-only JOIN with planes and airlines completed successfully.


# %% [markdown]

 ## Closing the Database Connection

 As a good practice, we should always close the database connection once all queries are executed.
 This ensures that any system resources associated with the connection are properly released.

In [ ]:
# %%

conn.close()
print(" Database connection closed successfully.")

 Database connection closed successfully.


# %% [markdown]

 ## Conclusion

 This notebook successfully replicated 16 SQL queries using pandas, demonstrating key data wrangling skills.
 The tasks included filtering, aggregation, grouping, and table joining — all performed using pandas syntax.
 For each SQL query, an equivalent pandas implementation was provided and validated using `assert_frame_equal`
 to ensure accuracy.

 This exercise strengthened my ability to translate traditional SQL logic into efficient, Pythonic data processing workflows
 using pandas, which is a valuable skill for real-world data analysis tasks.